## Multi-Agent Session Example

Join multiple agents to a session and have them return a chain response.

In [4]:
import os

os.environ["LOG_LEVEL"] = "WARNING"
import json
import asyncio
from gai.sessions import SessionManager
from rich.console import Console

console = Console(force_terminal=True)
from gai.messages.typing import MessagePydantic
from gai.sessions.operations.chat import ChatResponder
from gai.lib.tests import make_local_tmp
from gai.messages.dialogue import Dialogue

# Helper function to create a node that returns mock agent responses

async def create_node(node_name: str, session_mgr: SessionManager, plan):

    async def input_chunks_handler(pydantic: MessagePydantic):
        if pydantic.body.type == "chat.reply":
            # Should never handle reply messages.
            raise ValueError("input_chunks_callback should not process reply messages.")

        # Return a simulated streamer
        from gai.asm.agents import ToolUseAgent
        from gai.lib.config import config_helper
        llm_config = config_helper.get_client_config("sonnet-4")
        agent = ToolUseAgent(
            agent_name=node_name,
            llm_config=llm_config
        )
        
        recap = session_mgr.dialogue.extract_recap()        

        # Time to call chat completion
        async def get_streamer():
            from gai.asm.agents.tool_use_agent import AutoResumeError
            resp=agent.start(user_message=pydantic.body.content,recap=recap)
            content=""
            async for chunk in resp:
                if isinstance(chunk,str):
                    if chunk:
                        content += chunk
                yield chunk
            content+="\n"
            try:
                resp = agent.resume()
                async for chunk in resp:
                    if isinstance(chunk, str):
                        if chunk:
                            content += chunk
                        yield chunk
            except AutoResumeError:
                # conversation is over.
                if not content:
                    content = "My task is completed and I have nothing to resume from."
                session_mgr.dialogue.add_user_message(
                    recipient=agent.fsm.agent_name, content=pydantic.body.content
                )
                session_mgr.dialogue.add_assistant_message(
                    sender=agent.fsm.agent_name, chunk="<eom>", content=content
                )
                return

        return get_streamer()

    async def completed_content_handler(pydantic: MessagePydantic):
        session_mgr.log_message(pydantic)

    node = ChatResponder(node_name=node_name, session_mgr=session_mgr)
    await node.subscribe(
        input_chunks_callback=input_chunks_handler,
        completed_content_callback=completed_content_handler,
    )
    node.plans[plan.dialogue_id] = plan.model_copy()
    return node


# Helper function for displaying the results


def print_title(title):
    console.print(
        f"""\n[bold bright_yellow]{len(title) * "="}\n{title}\n{len(title) * "="}[/bold bright_yellow]\n"""
    )


def show_messages(session_mgr):
    print("The output below shows the messages saved in dialogue.messages:")
    for message in session_mgr.dialogue._messages:
        print(f"ID: {message.id}")
        print(f"  Sender: {message.header.sender}")
        print(f"  Recipient: {message.header.recipient}")
        print(f"Body:\n{json.dumps(message.body.model_dump(), indent=4)}")
        print("\n" + "-" * 10 + "\n")


In [2]:
import os
import asyncio
from gai.sessions.operations.chat import ChatSender
from rich.console import Console
from gai.lib.tests import make_local_tmp
from gai.sessions import SessionManager

console = Console(force_terminal=True)

# Get the current directory

here = os.getcwd()

# Initialize the dialogue
app_dir = make_local_tmp()
session_mgr = SessionManager(file_path=os.path.join(app_dir, "dialogue.json"))
await session_mgr.start()

## Create plan

from gai.sessions.operations.handshake import HandshakeSender

plan = HandshakeSender.create_plan("""
    User ->> Sara
    Sara ->> Diana
    """)

## Register Sara and wait for User

sara = await create_node("Sara", session_mgr, plan)

## Register Diana and wait for User

diana = await create_node("Diana", session_mgr, plan)

# Create an output message queue for agent response

a_queue = asyncio.Queue()

async def output_chunks_handler(pydantic: MessagePydantic):
    a_queue.put_nowait(pydantic)

# Register the User

user = ChatSender(node_name="User", session_mgr=session_mgr)

await user.subscribe(output_chunks_callback=output_chunks_handler)

async def stream_response(a_queue):
    chunk = await a_queue.get()

    while chunk.body.chunk != "<eom>":
        if chunk.body.chunk_no == 0:
            console.print(f"[bright_green]{chunk.header.sender}[/bright_green] :")
        if isinstance(chunk.body.chunk, str):
            print(chunk.body.chunk, end="", flush=True)
            chunk = await a_queue.get()

    return len(user.plan.steps) - 1 > user.plan.curr_step_no

## START Chain Response

step=await user.chat_send(user_message="Tell me a one paragraph story about a dragon and a knight.", plan=plan)
can_resume = await stream_response(a_queue)
while can_resume:
    await user.next()
    can_resume = await stream_response(a_queue)


Sara :

In the mist-shrouded mountains of Valdris, Sir Elara discovered that the fearsome dragon terrorizing nearby villages was actually guarding a clutch of orphaned eggs from poachers. Instead of drawing her sword, she approached with open hands and learned that Pyrion, the ancient wyrm, had lost his mate to treasure hunters and was desperately protecting the last of his kind. Understanding his pain, Elara proposed an alliance - she would help relocate the eggs to a safer, more remote sanctuary, while Pyrion would cease his raids on the villages. Together, they spent three days carefully transporting each precious egg to a hidden cave deep within the Whispering Peaks, and when their task was complete, Pyrion gifted Elara a single iridescent scale as a token of their unlikely friendship, proving that sometimes the greatest victories come not from conquest, but from compassion.


Diana :

What a beautiful story Sara told! I love how she turned the traditional dragon-slaying tale into something about understanding and cooperation. The image of Sir Elara and Pyrion carefully carrying those precious eggs to safety really captures the heart of it.

It makes me think about how many conflicts in our own world might be resolved if we took the time to understand what's really driving someone's actions, rather than just seeing the surface behavior. Pyrion wasn't evil - he was grieving and protecting what little family he had left.

What did you think of Sara's story? Did the twist surprise you, or were you expecting something different when you asked for a dragon and knight tale?


In [5]:
show_messages(session_mgr)


The output below shows the messages saved in dialogue.messages:
ID: adb8126c-0fb8-4a20-90bd-daa871b8d65d
  Sender: User
  Recipient: Sara
Body:
{
    "type": "chat.send",
    "dialogue_id": "00000000-0000-0000-0000-000000000000",
    "round_no": 0,
    "step_no": 0,
    "message_id": "00000000-0000-0000-0000-000000000000.300",
    "content_type": "text",
    "role": "user",
    "content": "Sara, Tell me a one paragraph story about a dragon and a knight."
}

----------

ID: f80d93d9-0158-494f-a029-d415de2f1689
  Sender: User
  Recipient: Sara
Body:
{
    "type": "chat.send",
    "dialogue_id": "00000000-0000-0000-0000-000000000000",
    "round_no": 1,
    "step_no": 0,
    "message_id": "00000000-0000-0000-0000-000000000000.301",
    "content_type": "text",
    "role": "user",
    "content": "Sara, Tell me a one paragraph story about a dragon and a knight."
}

----------

ID: 53e9622c-5bde-446c-965f-2f737af3d6b4
  Sender: User
  Recipient: Sara
Body:
{
    "type": "chat.send",
    "dial